https://www.data.go.kr/data/15097922/fileData.do

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
page = '2'
document_num = '471168842'
url = 

url

In [ ]:
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
#403오류 해결

comment_list = []

if (response.status_code == 200):
    html = response.text
    soup = bs(html, 'html.parser')
    selector = 'div.exJsOriginalContent > div.repCnt'
    for i  in soup.select(selector):
        comment_list.append(i)
    print(comment_list)
    
else:
    print(f'오류: {response.status_code}')